# Portfolio Experiments

In [1]:
import yaml, numpy as np, pandas as pd

def load_config(path='default_config.yaml', **ov):
    with open(path) as f: cfg = yaml.safe_load(f)
    cfg.update(ov)
    return cfg

In [2]:
cfg = load_config()  # or: load_config(num_epochs=50)

In [3]:
# =============================================================================
# Cell 2: Data Loading & Cross-Validation Splits
# =============================================================================
# Expanding window validation for financial time series
# Avoids look-ahead bias and tests across multiple market regimes
# Reference: de Prado (2018) "Advances in Financial Machine Learning", Ch. 7

def create_folds(data, test_years=2, init_train_years=10, val_years=1, step_years=1):
    """
    Expanding window cross-validation for time series.
    
    Returns:
        folds: list of (train_df, val_df) tuples
        test: held-out test set (final test_years of data)
    """
    days_per_year = 252  # trading days
    
    # Hold out final test set
    test_size = test_years * days_per_year
    test = data.iloc[-test_size:]
    remaining = data.iloc[:-test_size]
    
    # Create expanding folds
    folds = []
    train_end = init_train_years * days_per_year
    val_size = val_years * days_per_year
    step = step_years * days_per_year
    
    while train_end + val_size <= len(remaining):
        train = remaining.iloc[:train_end]
        val = remaining.iloc[train_end:train_end + val_size]
        folds.append((train, val))
        train_end += step
    
    return folds, test

data = pd.read_csv('data.csv', index_col=0, parse_dates=True)
folds, test = create_folds(data)
print(f"Created {len(folds)} folds, test set: {len(test)} days")

Created 5 folds, test set: 504 days


In [4]:
# =============================================================================
# Cell 3: Equal Weights Baseline
# =============================================================================
# 1/N portfolio - surprisingly hard to beat (DeMiguel et al., 2009)
# Reference: DeMiguel, Garlappi & Uppal (2009) "Optimal Versus Naive 
#            Diversification", Review of Financial Studies

w = np.ones(test.shape[1]) / test.shape[1]
r = (test * w).sum(axis=1)
ew_sharpe = r.mean() / r.std() * np.sqrt(252)
print(f"Equal Weights Sharpe: {ew_sharpe:.3f}")

Equal Weights Sharpe: 1.645


In [5]:
# =============================================================================
# Cell 4: DDPG / DDES-DDPG Runner
# =============================================================================
# DDPG: Lillicrap et al. (2015) "Continuous control with deep reinforcement learning"
# DDES: Hong et al. (2018) "Diversity-Driven Exploration Strategy for Deep RL"
#
# Key difference:
#   DDPG  - exploration via Gaussian noise on actions
#   DDES  - exploration via diversity term in actor loss: -Q(s,a) + α * D(a, a_prior)

from models.ddpg import DDPG
from models.networks import NeuralNetwork

def run_ddpg(train, val, test, cfg, use_ddes=False):
    """Train and evaluate DDPG or DDES-DDPG."""
    agent = DDPG(
        lookback_window=cfg['lookback_window'],
        predictor=NeuralNetwork,
        batch_size=cfg['batch_size'],
        hidden_sizes=cfg['hidden_sizes'],
        seed=cfg['seeds'][0],
    )
    agent.train(
        train, val,
        actor_lr=cfg['actor_lr'],
        critic_lr=cfg['critic_lr'],
        gamma=cfg['gamma'],
        tau=cfg['tau'],
        soft_update=cfg['soft_update'],
        num_epochs=cfg['num_epochs'],
        patience=cfg['patience'],
        noise=cfg['noise'],
        use_ddes=use_ddes,
        ddes_alpha=cfg['ddes_alpha'],
    )
    _, (_, sharpe) = agent.evaluate(test)
    return sharpe

In [6]:
# =============================================================================
# Cell 5: Run Experiments Across Folds
# =============================================================================
results = {'EW': ew_sharpe, 'DDPG': [], 'DDES': []}

for i, (train, val) in enumerate(folds):
    print(f"\n--- Fold {i+1}/{len(folds)} ---")
    print(f"Train: {len(train)} days, Val: {len(val)} days")
    
    results['DDPG'].append(run_ddpg(train, val, test, cfg, use_ddes=False))
    results['DDES'].append(run_ddpg(train, val, test, cfg, use_ddes=True))


--- Fold 1/5 ---
Train: 2520 days, Val: 252 days


TypeError: DDPG.train() got an unexpected keyword argument 'noise'